In [1]:
from pyspark.sql import SparkSession

In [2]:
def printConfigs(spark):
    mconf = spark.sparkContext.getConf().getAll()
    for conf in mconf:
        print(conf)


In [3]:
spark = ( SparkSession.builder 
    .appName("SparkConf and Optimization")
    .config("spark.sql.shuffle.partitions", 15)
    .config("spark.executor.memory", "2g")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .getOrCreate()
)

23/05/22 12:50:26 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/22 12:50:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 12:50:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/22 12:50:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
printConfigs(spark)

('spark.executor.memory', '2g')
('spark.sql.adaptive.enabled', 'true')
('spark.executor.id', 'driver')
('spark.driver.host', '192.168.1.44')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.sql.adapt

In [5]:
spark.sparkContext.defaultParallelism

4

actually, it doesn't work, we can't change the values at runtime exec

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

In [7]:
print(" ***** Setting spark.sql.shuffle.partitions to defaultParallelism ***** ")

 ***** Setting spark.sql.shuffle.partitions to defaultParallelism ***** 


In [8]:
printConfigs(spark)

('spark.executor.memory', '2g')
('spark.sql.adaptive.enabled', 'true')
('spark.executor.id', 'driver')
('spark.driver.host', '192.168.1.44')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.sql.adapt

----

In [9]:
spark.sparkContext.getConf().get("spark.driver.host")

'192.168.1.44'

In [10]:
def printConfigSpec(spark, key):
    print(spark.sparkContext.getConf().get(key))

In [11]:
printConfigSpec(spark, "spark.driver.host")
printConfigSpec(spark, "spark.driver.port")
printConfigSpec(spark, "spark.app.name")
printConfigSpec(spark, "spark.master")
printConfigSpec(spark, "spark.submit.deployMode")
printConfigSpec(spark, "spark.app.id")

192.168.1.44
33349
SparkConf and Optimization
local[*]
client
local-1684770628106


----

Spark SQL–specific Spark configs

In [12]:
spark.sql("SET -v").select("key", "value").show(n=5, truncate=False)

+---------------------------------------------------------+----------------------------------------------------------------+
|key                                                      |value                                                           |
+---------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes          |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.autoBroadcastJoinThreshold            |<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.enabled            |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum|<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionSize   |1MB                                                             |


In [13]:
spark.conf.isModifiable("spark.sql.adaptive.coalescePartitions.initialPartitionNum")

True

In [14]:
spark.conf.set("spark.sql.adaptive.coalescePartitions.initialPartitionNum", 4)

In [15]:
spark.sql("SET -v").select("key", "value").show(n=5, truncate=False)

+---------------------------------------------------------+----------------------------------------------------------------+
|key                                                      |value                                                           |
+---------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes          |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.autoBroadcastJoinThreshold            |<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.enabled            |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum|4                                                               |
|spark.sql.adaptive.coalescePartitions.minPartitionSize   |1MB                                                             |


In [16]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [17]:
spark.conf.get("spark.sql.shuffle.partitions")

'5'

In [18]:
printConfigs(spark)

('spark.executor.memory', '2g')
('spark.sql.adaptive.enabled', 'true')
('spark.executor.id', 'driver')
('spark.driver.host', '192.168.1.44')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.sql.adapt

----
#### Optimizing

##### Repartition

In [19]:
repartDF = spark.range(100 * 100 * 100).repartition(16)

In [20]:
repartDF.rdd.getNumPartitions()

16

##### Cache

In [21]:
from pyspark.sql.functions import col

df = spark.range(1 * 10000000).toDF("id").withColumn("square", col("id") * col("id"))

In [22]:
df.cache() # Cache the data
df.count() # Materialize the cache

10000000

In [23]:
df.count()

10000000

3.3s vs 0.1s

##### Persist

In [24]:
from pyspark.storagelevel import StorageLevel

In [25]:
df_p = spark.range(1 * 10000001).toDF("id").withColumn("square", col("id") * col("id"))

In [26]:
df_p.persist(StorageLevel.DISK_ONLY) # Serialize the data and cache it on disk
df_p.count() # Materialize the cache

10000001

In [27]:
df_p.count()

10000001

2.5s vs 0.2s

##### cache the tables or views derived from DataFrames.

In [28]:
df.createOrReplaceTempView("dfTable")
spark.sql("CACHE TABLE dfTable")
spark.sql("SELECT count(*) FROM dfTable").show()

23/05/22 12:50:43 WARN CacheManager: Asked to cache already cached data.


+--------+
|count(1)|
+--------+
|10000000|
+--------+

